In [1]:
import os
 
def file_name(file_dir): 
    for root, dirs, files in os.walk(file_dir):
        continue
        #print(root) #当前目录路径
        #print(dirs) #当前路径下所有子目录
       # print(files) #当前路径下所有非目录子文件
    return files


## 控制c 只有需要检测的图片

# 加载模型

In [2]:
import argparse
import os
import torch
import torch.backends.cudnn as cudnn
import numpy as np
import PIL.Image as pil_image

from myutils import rgb2ycbcr, ycbcr2rgb, calc_psnr

In [3]:
cudnn.benchmark = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
from rcan import make_model

In [5]:
class modelargs():
    def __init__(self):
        self.n_resgroups=10
        self.n_resblocks=20
        self.n_feats=64
        self.reduction=16
        self.scale=[2]
        self.rgb_range=255
        self.n_colors=3
        self.res_scale=1

In [6]:
model = make_model(modelargs()).to(device)

# 定义psnr函数

In [7]:
def get_psnr(testdata,myscale,picidx,j):
   # print(filename)
    scale=myscale
   # text= os.path.split(filename)[-1]
  #  textname = os.path.splitext(text)[0]
    
    lrpath="../LR/{}/x{}/".format(testdata,myscale)
    hrpath="../HR/{}/x{}/".format(testdata,myscale)
    
    clrpath=file_name(lrpath)
    chrpath=file_name(hrpath)
  #  print(clrpath[picidx])
    textname=str(clrpath[picidx]).split("_LRBI")[0]
#     textname=textnames[0]
#     for i in range(len(textnames)-1):
#         if(textnames[i+1]=='LRBI'):
#             break
#         textname=textname+textnames[i+1]
        
        
    
    
    
    lr = pil_image.open(lrpath+clrpath[picidx]).convert('RGB')
    lr = np.array(lr).astype(np.float32).transpose([2,0,1])
 #   lrycbcr = rgb2ycbcr(lr)
    
   # y = lrycbcr[..., 0]
   # print(lr.shape)
    y =lr /255.
    y = torch.from_numpy(y).to(device)
    y = y.unsqueeze(0)
    
    
    #print("y:{}".format(y.size()))########################################################################

    hr = pil_image.open(hrpath+chrpath[picidx]).convert('RGB')
    hr=np.array(hr).astype(np.float32).transpose([2,0,1])
  #  hrycbcrb = rgb2ycbcr(hr)
  #  ya = hrycbcrb[..., 0]
  
    ya =hr/ 255.
    ya = torch.from_numpy(ya).to(device)
    ya = ya.unsqueeze(0)
  #  print("ya:{}".format(ya.size()))########################################################################

    #print(y.size())

    with torch.no_grad():
        preds = model(y).clamp(0.0, 1.0)

    psnr = calc_psnr(ya, preds)

    print('PSNR: {:.2f}'.format(psnr))

    preds = preds.mul(255.0).cpu().numpy().squeeze(0)
    
    #print(preds.shape)
    output = np.array(preds).transpose([1, 2, 0])
    output = np.clip((output), 0.0, 255.0).astype(np.uint8)
    output = pil_image.fromarray(output)
    savepath='./save/{}/{}/x{}/{}_idn_x{}.png'.format(j,testdata,myscale,textname,scale)
    
    if not os.path.exists(os.path.split(savepath)[0]):
        os.makedirs(os.path.split(savepath)[0])
     
    
    output.save(savepath)
    
    preds = None
    output = None
    ya = None
    y = None
    lr = None
    hr = None
    
    
    
    return psnr


# 单独计算psnr

In [8]:
modelname='./checkpoint/model_epoch_900.pth'
print(modelname)

./checkpoint/model_epoch_900.pth


In [9]:
from rcan import make_model

In [10]:
class modelargs():
    def __init__(self):
        self.n_resgroups=10
        self.n_resblocks=20
        self.n_feats=64
        self.reduction=16
        self.scale=[2]
        self.rgb_range=255
        self.n_colors=3
        self.res_scale=1

In [11]:
model = make_model(modelargs()).to(device)

In [12]:
allpsnr=[]

In [13]:
for j in range(1000):
    modelname='./checkpoint/model_epoch_{}.pth'.format(j+1)
    print(modelname)
    state_dict = model.state_dict()
    for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

    testdata="Set5"
    myscale = 2
    hrpath="../HR/{}/x{}/".format(testdata,myscale)
    lrpath="../LR/{}/x{}/".format(testdata,myscale)
    clrpath=file_name(lrpath)
    count=len(clrpath)

    hrpath="../HR/{}/x{}/".format(testdata,myscale)

    chrpath=file_name(hrpath)

    count=len(chrpath)
    print(count)
    model=model.to(device)
    model.eval()
    sum_psnr=0
    for i in range(count):
     #   filename = os.path.join("../"+testdata+"/"+c[i])

        sum_psnr += get_psnr(testdata,myscale,i,j)

    print("avg_psnr is :{:.2f}".format(sum_psnr/count))
    allpsnr.append(sum_psnr/count)

./checkpoint/model_epoch_1.pth
5
PSNR: 15.86
PSNR: 15.59
PSNR: 13.10
PSNR: 18.07
PSNR: 15.71
avg_psnr is :15.67
./checkpoint/model_epoch_2.pth
5
PSNR: 16.10
PSNR: 18.29
PSNR: 14.98
PSNR: 18.00
PSNR: 15.96
avg_psnr is :16.66
./checkpoint/model_epoch_3.pth
5
PSNR: 20.68
PSNR: 20.31
PSNR: 17.40
PSNR: 21.43
PSNR: 20.19
avg_psnr is :20.00
./checkpoint/model_epoch_4.pth
5
PSNR: 23.20
PSNR: 20.95
PSNR: 18.27
PSNR: 23.66
PSNR: 22.21
avg_psnr is :21.66
./checkpoint/model_epoch_5.pth
5
PSNR: 24.97
PSNR: 23.11
PSNR: 19.61
PSNR: 25.33
PSNR: 23.62
avg_psnr is :23.33
./checkpoint/model_epoch_6.pth
5
PSNR: 25.14
PSNR: 23.73
PSNR: 20.25
PSNR: 25.57
PSNR: 23.53
avg_psnr is :23.64
./checkpoint/model_epoch_7.pth
5
PSNR: 26.09
PSNR: 24.42
PSNR: 20.85
PSNR: 26.44
PSNR: 24.93
avg_psnr is :24.55
./checkpoint/model_epoch_8.pth
5
PSNR: 27.28
PSNR: 24.82
PSNR: 21.41
PSNR: 27.04
PSNR: 25.54
avg_psnr is :25.22
./checkpoint/model_epoch_9.pth
5
PSNR: 27.67
PSNR: 25.32
PSNR: 21.82
PSNR: 27.27
PSNR: 25.82
avg_psnr is

PSNR: 28.21
PSNR: 31.61
PSNR: 32.48
avg_psnr is :32.75
./checkpoint/model_epoch_146.pth
5
PSNR: 35.95
PSNR: 34.33
PSNR: 28.11
PSNR: 31.56
PSNR: 32.36
avg_psnr is :32.46
./checkpoint/model_epoch_147.pth
5
PSNR: 36.12
PSNR: 35.15
PSNR: 28.20
PSNR: 31.51
PSNR: 32.41
avg_psnr is :32.68
./checkpoint/model_epoch_148.pth
5
PSNR: 36.51
PSNR: 35.59
PSNR: 28.32
PSNR: 31.73
PSNR: 32.58
avg_psnr is :32.95
./checkpoint/model_epoch_149.pth
5
PSNR: 35.23
PSNR: 34.95
PSNR: 28.16
PSNR: 31.29
PSNR: 32.11
avg_psnr is :32.35
./checkpoint/model_epoch_150.pth
5
PSNR: 35.94
PSNR: 35.74
PSNR: 28.29
PSNR: 31.66
PSNR: 32.47
avg_psnr is :32.82
./checkpoint/model_epoch_151.pth
5
PSNR: 35.09
PSNR: 34.97
PSNR: 28.09
PSNR: 31.35
PSNR: 31.85
avg_psnr is :32.27
./checkpoint/model_epoch_152.pth
5
PSNR: 35.82
PSNR: 35.69
PSNR: 28.27
PSNR: 31.63
PSNR: 32.26
avg_psnr is :32.73
./checkpoint/model_epoch_153.pth
5
PSNR: 36.30
PSNR: 35.66
PSNR: 28.31
PSNR: 31.71
PSNR: 32.52
avg_psnr is :32.90
./checkpoint/model_epoch_154.pth


./checkpoint/model_epoch_289.pth
5
PSNR: 36.81
PSNR: 37.26
PSNR: 29.57
PSNR: 31.96
PSNR: 33.53
avg_psnr is :33.82
./checkpoint/model_epoch_290.pth
5
PSNR: 36.87
PSNR: 37.21
PSNR: 29.55
PSNR: 31.98
PSNR: 33.59
avg_psnr is :33.84
./checkpoint/model_epoch_291.pth
5
PSNR: 36.82
PSNR: 37.22
PSNR: 29.58
PSNR: 31.97
PSNR: 33.54
avg_psnr is :33.83
./checkpoint/model_epoch_292.pth
5
PSNR: 36.79
PSNR: 37.21
PSNR: 29.58
PSNR: 31.92
PSNR: 33.48
avg_psnr is :33.80
./checkpoint/model_epoch_293.pth
5
PSNR: 36.57
PSNR: 37.19
PSNR: 29.54
PSNR: 31.93
PSNR: 33.44
avg_psnr is :33.74
./checkpoint/model_epoch_294.pth
5
PSNR: 36.91
PSNR: 37.31
PSNR: 29.63
PSNR: 31.98
PSNR: 33.59
avg_psnr is :33.88
./checkpoint/model_epoch_295.pth
5
PSNR: 36.69
PSNR: 37.28
PSNR: 29.61
PSNR: 31.95
PSNR: 33.52
avg_psnr is :33.81
./checkpoint/model_epoch_296.pth
5
PSNR: 36.78
PSNR: 37.31
PSNR: 29.62
PSNR: 31.97
PSNR: 33.57
avg_psnr is :33.85
./checkpoint/model_epoch_297.pth
5
PSNR: 36.82
PSNR: 37.25
PSNR: 29.64
PSNR: 31.97
PSNR:

FileNotFoundError: [Errno 2] No such file or directory: './checkpoint/model_epoch_401.pth'

In [14]:
len(allpsnr)

400

In [15]:
mymax=0
mymaxi=-1
for i in range(len(allpsnr)):
    if(allpsnr[i]>mymax):
        mymax=allpsnr[i]
        mymaxi=i

In [16]:
mymax

tensor(34.1863, device='cuda:0')

In [17]:
mymaxi

399

In [18]:
988

988

In [20]:
modelname='./checkpoint/model_epoch_900.pth'
print(modelname)
state_dict = model.state_dict()
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
    if n in state_dict.keys():
        state_dict[n].copy_(p)
    else:
        raise KeyError(n)

testdata="Set5"
myscale = 2
hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model=model.to(device)
model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i,j)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))
allpsnr.append(sum_psnr/count)

./checkpoint/model_epoch_900.pth
5
PSNR: 37.29
PSNR: 39.57
PSNR: 32.49
PSNR: 32.26
PSNR: 34.65
avg_psnr is :35.25


In [41]:
testdata="Manga109"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

109


KeyboardInterrupt: 

In [16]:
testdata="Set14"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

14
PSNR: 23.12
PSNR: 27.00
PSNR: 27.34
PSNR: 28.94
PSNR: 25.91
PSNR: 32.00
PSNR: 29.88
PSNR: 32.38
PSNR: 33.32
PSNR: 28.82
PSNR: 33.12
PSNR: 30.91
PSNR: 26.87
PSNR: 31.23
avg_psnr is :29.35


In [17]:
testdata="B100"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()

sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

100
PSNR: 25.14
PSNR: 29.78
PSNR: 27.52
PSNR: 32.94
PSNR: 28.34
PSNR: 34.79
PSNR: 29.47
PSNR: 25.07
PSNR: 29.97
PSNR: 31.78
PSNR: 25.92
PSNR: 31.14
PSNR: 33.48
PSNR: 30.69
PSNR: 27.18
PSNR: 28.57
PSNR: 35.61
PSNR: 34.27
PSNR: 26.61
PSNR: 27.38
PSNR: 24.26
PSNR: 25.91
PSNR: 25.99
PSNR: 28.27
PSNR: 30.22
PSNR: 29.83
PSNR: 29.91
PSNR: 32.44
PSNR: 32.06
PSNR: 21.30
PSNR: 32.24
PSNR: 22.62
PSNR: 28.23
PSNR: 25.71
PSNR: 35.50
PSNR: 29.00
PSNR: 28.61
PSNR: 29.11
PSNR: 31.04
PSNR: 35.51
PSNR: 28.17
PSNR: 26.86
PSNR: 28.02
PSNR: 29.58
PSNR: 25.95
PSNR: 35.60
PSNR: 24.98
PSNR: 27.21
PSNR: 29.04
PSNR: 33.51
PSNR: 26.42
PSNR: 25.19
PSNR: 33.45
PSNR: 34.04
PSNR: 29.94
PSNR: 27.31
PSNR: 22.48
PSNR: 28.88
PSNR: 33.36
PSNR: 31.61
PSNR: 32.99
PSNR: 30.41
PSNR: 35.47
PSNR: 26.59
PSNR: 27.64
PSNR: 30.68
PSNR: 39.54
PSNR: 23.32
PSNR: 24.08
PSNR: 30.93
PSNR: 30.95
PSNR: 27.76
PSNR: 29.24
PSNR: 27.70
PSNR: 25.54
PSNR: 32.07
PSNR: 25.47
PSNR: 31.87
PSNR: 31.52
PSNR: 31.74
PSNR: 37.45
PSNR: 29.39
PSNR: 23.88


In [18]:
testdata="Urban100"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)


model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

100
PSNR: 28.69
PSNR: 27.44
PSNR: 25.46
PSNR: 23.44
PSNR: 25.97
PSNR: 24.22
PSNR: 28.35
PSNR: 23.33
PSNR: 33.23
PSNR: 27.56
PSNR: 18.08
PSNR: 25.85
PSNR: 28.91
PSNR: 26.59
PSNR: 27.13
PSNR: 28.48
PSNR: 26.20
PSNR: 27.59
PSNR: 22.67
PSNR: 23.22
PSNR: 30.14
PSNR: 29.28
PSNR: 29.22
PSNR: 20.77
PSNR: 29.55
PSNR: 29.32
PSNR: 30.50
PSNR: 31.43
PSNR: 26.27
PSNR: 24.17
PSNR: 26.31
PSNR: 29.04
PSNR: 27.69
PSNR: 24.60
PSNR: 26.88
PSNR: 28.23
PSNR: 25.81
PSNR: 28.11
PSNR: 25.15
PSNR: 22.29
PSNR: 23.66
PSNR: 27.50
PSNR: 25.59
PSNR: 27.99
PSNR: 23.63
PSNR: 26.31
PSNR: 23.66
PSNR: 21.56
PSNR: 23.50
PSNR: 26.75
PSNR: 27.36
PSNR: 26.70
PSNR: 23.48
PSNR: 23.60
PSNR: 26.58
PSNR: 25.09
PSNR: 28.98
PSNR: 25.06
PSNR: 24.59
PSNR: 24.78
PSNR: 25.91
PSNR: 22.91
PSNR: 20.60
PSNR: 27.77
PSNR: 26.26
PSNR: 25.34
PSNR: 20.06
PSNR: 29.79
PSNR: 26.13
PSNR: 26.44
PSNR: 29.06
PSNR: 20.02
PSNR: 22.05
PSNR: 25.38
PSNR: 29.76
PSNR: 23.66
PSNR: 26.30
PSNR: 28.70
PSNR: 26.94
PSNR: 34.99
PSNR: 31.08
PSNR: 30.27
PSNR: 24.17


In [13]:
myscale = 3
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

model.eval()
sum_psnr=0
for i in range(count):
   
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("psnr is :{:.2f}".format(sum_psnr/count))

PSNR: 30.63
PSNR: 25.66
PSNR: 34.00
PSNR: 33.36
PSNR: 27.44
PSNR: 28.82
PSNR: 27.34
PSNR: 27.23
PSNR: 26.26


KeyboardInterrupt: 

In [ ]:
myscale = 4
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

model.eval()
sum_psnr=0
for i in range(count):
      
    sum_psnr += get_psnr(testdata,myscale,i)
print("psnr is :{:.2f}".format(sum_psnr/count))